# Exploring the K matrix!
## In this notebook, we'll look a bit more closely at the K matrix and what it means
### Topics we will cover:
#### How to generate a K matrix
#### What different formulations entail
#### What does this look like in maize vs arabidopsis?

# 1.  Initial setup steps

## 1a. Prepare environment
Loading packages and functions into R

In [ ]:
library(sommer) #Mixed effects models package
library(rTASSEL)
library(plot.matrix)
options(repr.plot.width=12, repr.plot.height=5)## this sets a larger size for figures

## 1b. Define input variables

In [ ]:
# genotype data for maize and arabidopsis (in the "hdf5" format)
default.par <- par()
zmG <- readGenotypeTableFromPath("./data/282.poly_thinned30kbp.h5")
atG <- readGenotypeTableFromPath("./data/1001genomes_snp-short-indel_only_ACGTN.subsamp170_poly_minCov50_thinned30kpb.h5")
# phenotype data for maize and arabidopsis
zmP <- readPhenotypeFromPath("./data/282_traits.txt")
atP <- readPhenotypeFromPath("./data/Arabidopsis_Phenotypes.trait")
# summary info for maize and arabidopsis 
zmSS <- read.table("./data/282.poly_thinned30kbp_SiteSummary.txt",header=T,as.is=T,sep="\t")
atSS <- read.table("./data/1001genomes_snp-short-indel_only_ACGTN.subsamp170_poly_minCov50_thinned30kpb_SiteSummary.txt",header=T,as.is=T,sep="\t")
zmTS <- read.table("./data/282.poly_thinned30kbp_TaxaSummary.txt",header=T,as.is=T,sep="\t")
atTS <- read.table("./data/1001genomes_snp-short-indel_only_ACGTN.subsamp170_poly_minCov50_thinned30kpb_TaxaSummary.txt",header=T,as.is=T,sep="\t")
taxa <- c("P39:250040828","Pa762:250040188","Pa875:250039820","NC230:250047946","NC232:250040051","NC236:250040160")

# 2.  Generate K (kinship/genetic similarity) matrices for a small subset to compare
### We will calculate these in two different ways, each with different assumptions regarding population expectations for inbreeding
### "Centered" assumes Hardy-Weinburg and is calculated after J. Yang, S. H. Lee, M. E. Goddard, P. M. Visscher, GCTA: a tool for genome-wide complex trait analysis. Am. J. Hum. Genet. 88, 76–82 (2011).

In [ ]:
# Centered K matrix
centered <- kinshipMatrix(filterGenotypeTableTaxa(zmG,taxa = taxa),method = "Centered_IBS")
plot(kinshipToRMatrix(centered),main="centered K",digits = 3,las=2,breaks = 36)

### "Normalized" allows for inbreeding and is calculated after J. B. Endelman, J.-L. Jannink, Shrinkage estimation of the realized relationship matrix. G3 . 2, 1405–1413 (2012).

In [ ]:
# Normalized K matrix
normalized <- kinshipMatrix(filterGenotypeTableTaxa(zmG,taxa = taxa),method = "Normalized_IBS")
plot(kinshipToRMatrix(normalized),main="normalized K",digits = 3,las=2,breaks = 36)
par(default.par)

### Both of these differ from an "IBS Distance Matrix" in that they are similarity matrices that account for population allele frequencies. Why does this matter?

In [ ]:
# IBS Distance matrix
ibsDist <- distanceMatrix(filterGenotypeTableTaxa(zmG,taxa = taxa))
plot(distanceToRMatrix(ibsDist),main="IBS distance",digits = 3,las=2,breaks = 36)

# 3.  A special property of the normalized K matrix is that the mean of the diagonal (the "trace", representing the relationship of each line to itself) equals 1 + f. What is it for this population?

In [ ]:
mean(diag(kinshipToRMatrix(normalized)))

# 4.  What happens when you subset the individuals included? Do the values change? Why?

In [ ]:
# With only three individuals
taxa <- c("P39:250040828","Pa875:250039820","NC232:250040051")
# IBS Distance matrix
ibsDist <- distanceMatrix(filterGenotypeTableTaxa(zmG,taxa = taxa))
plot(distanceToRMatrix(ibsDist),main="IBS distance",digits = 3,las=2,breaks = 36)
# Centered K matrix
centered <- kinshipMatrix(filterGenotypeTableTaxa(zmG,taxa = taxa),method = "Centered_IBS")
plot(kinshipToRMatrix(centered),main="centered K",digits = 3,las=2,breaks = 36)
# Normalized K matrix
normalized <- kinshipMatrix(filterGenotypeTableTaxa(zmG,taxa = taxa),method = "Normalized_IBS")
plot(kinshipToRMatrix(normalized),main="normalized K",digits = 3,las=2,breaks = 36)
mean(diag(kinshipToRMatrix(normalized)))
par(default.par)

# 5.  We'll look at all the individuals now. How do the diagonals differ?

In [ ]:
centered <- kinshipMatrix(zmG,method = "Centered_IBS")
normalized <- kinshipMatrix(zmG,method = "Normalized_IBS")
# Look at the diagonal as a histogram
hist(diag(kinshipToRMatrix(centered)),breaks = 30,xlab="Calculated relationship of each line to itself",main="Trace of centered K matrix",col = "firebrick")
hist(diag(kinshipToRMatrix(normalized)),breaks = 30,xlab="Calculated relationship of each line to itself",main="Trace of normalized K matrix",col = "blue")

## What is the heterozygosity for the population?

In [ ]:
# What is the heterozygosity for the population?
hist(zmTS$Proportion.Heterozygous,breaks=30,main="Heterozygosity in maize",xlab="Proportion heterozygous",col="darkgreen")

## check the inbreeding coefficient for the whole population

In [ ]:
mean(diag(kinshipToRMatrix(normalized)))

# 6. Let's look at the Arabidopsis population.

In [ ]:
centered <- kinshipMatrix(atG,method = "Centered_IBS")
normalized <- kinshipMatrix(atG,method = "Normalized_IBS")
# Look at the diagonal as a histogram
hist(diag(kinshipToRMatrix(centered)),breaks = 30,xlab="Calculated relationship of each line to itself",main="Trace of centered K matrix",col = "firebrick")
hist(diag(kinshipToRMatrix(normalized)),breaks = 30,xlab="Calculated relationship of each line to itself",main="Trace of normalized K matrix",col = "blue")

## What is the heterozygosity for the population?

In [ ]:
summary(atTS$Proportion.Heterozygous)

## check the inbreeding coefficient for the whole population

In [ ]:
mean(diag(kinshipToRMatrix(normalized)))

# 7. We know MAF is important for generating the K matrix. How does Arabidopsis compare to maize?

In [ ]:
# maize
hist(atSS$Minor.Allele.Frequency,col="#00ff0080",freq = F,main="Folded site frequency spectrum (SFS)",xlab="MAF")
hist(zmSS$Minor.Allele.Frequency,col="#ff000080",add=T,freq = F)
legend("topright",legend = c("arabidopsis","maize"),fill=c("#00ff0080","#ff000080"))

## What does this mean for calculating relatedness in the population?
